In [46]:
import numpy as np
import yfinance as yf
import pandas as pd
import methods.selenium_patch as selenium_patch
from datetime import timedelta
from methods.scraper import Ticker

In [47]:
ticker = "NVO"
data = Ticker(ticker).macrotrends_key_financial_ratios(frequency="quarterly")
display(data)

field_name,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,Inventory Turnover Ratio,Receiveable Turnover,Days Sales In Receivables,ROE - Return On Equity,Return On Tangible Equity,ROA - Return On Assets,ROI - Return On Investment,Book Value Per Share,Operating Cash Flow Per Share,Free Cash Flow Per Share
2025-09-30,0.77610,0.34420,0.59570,76.12040,31.58610,31.58610,,34.03360,26.68320,0.14640,0.37690,0.81170,110.88310,11.77540,31.60050,3.90520,7.72210,,1.6907,1.1821
2025-06-30,0.78400,0.34190,0.59060,83.28580,43.52110,43.52110,,43.98430,34.48350,0.15940,0.28140,0.89530,100.52270,15.76940,44.02490,5.49680,10.37820,,1.4558,0.9212
2025-03-31,0.73860,0.41010,0.85700,83.49280,49.67660,49.67660,,47.42530,37.18160,0.15960,0.30080,0.90410,99.54210,20.95710,99.56450,5.93550,12.36280,,0.78040,0.35440
2024-12-31,0.73970,0.38460,0.71640,84.80930,42.77470,42.77470,,41.36440,32.84000,0.18150,0.31450,0.96740,93.03670,19.35370,85.71980,5.96180,11.91020,4.65970,0.3395,-0.1665
2024-09-30,0.93670,0.29920,0.47270,84.14270,47.42890,47.42890,,48.21700,38.28440,0.17940,0.29800,0.89550,100.49800,22.65230,41.34390,6.86920,15.87510,,1.4961,1.0819
2024-06-30,0.94460,0.31440,0.50710,84.90450,38.10460,38.10460,,37.22010,29.45930,0.18430,0.28260,0.89400,100.67150,17.81870,35.16250,5.42800,12.21590,,1.6284,1.2922
2024-03-31,0.70300,0.14490,0.27220,84.82610,48.73220,48.73220,,48.84240,38.87890,0.21860,0.29510,0.91080,98.81560,25.68670,64.69660,8.49960,21.96410,,0.46620,0.19020
2023-12-31,0.82310,0.16150,0.25340,84.79770,40.63440,40.63450,,41.91990,33.33300,0.20860,0.31350,0.87170,103.24320,20.52240,47.38130,6.95380,17.20750,3.43070,0.2867,-0.0136
2023-09-30,0.82410,0.17650,0.28540,83.46190,45.82420,45.82420,,47.78230,38.27280,0.19570,0.32890,0.94780,94.95430,24.17220,60.31940,7.49010,19.90700,,1.3368,1.1464
2023-06-30,0.87830,0.17670,0.28410,85.53220,43.99260,43.99260,,44.66670,35.77900,0.19340,0.28040,0.77000,116.88560,21.47380,48.90750,6.92000,17.67980,,0.9428,0.7496


In [48]:
prices = yf.Ticker(ticker).history("max").get("Close")
earning_prices = []
for date in data.index:
    got_price = False
    i = 0
    while got_price == False and i<7:
        try:
            earning_prices.append(prices[pd.to_datetime(date).tz_localize("America/New_York") - timedelta(days=i)]) # type: ignore
            got_price = True
        except:
            i += 1

earning_changes = [np.nan]
for i in range(1, len(earning_prices)):
    earning_changes.append(earning_prices[i-1]/earning_prices[i])

In [49]:
data["Future Change%"] = earning_changes
data["Price"] = earning_prices
data = data[data.columns.to_list()[::-1]]
data

field_name,Price,Future Change%,Free Cash Flow Per Share,Operating Cash Flow Per Share,Book Value Per Share,ROI - Return On Investment,ROA - Return On Assets,Return On Tangible Equity,ROE - Return On Equity,Days Sales In Receivables,...,Asset Turnover,Net Profit Margin,Pre-Tax Profit Margin,EBITDA Margin,EBIT Margin,Operating Margin,Gross Margin,Debt/Equity Ratio,Long-term Debt / Capital,Current Ratio
2025-09-30,55.490002,NaN,1.1821,1.6907,,7.72210,3.90520,31.60050,11.77540,110.88310,...,0.14640,26.68320,34.03360,,31.58610,31.58610,76.12040,0.59570,0.34420,0.77610
2025-06-30,68.250916,0.813029,0.9212,1.4558,,10.37820,5.49680,44.02490,15.76940,100.52270,...,0.15940,34.48350,43.98430,,43.52110,43.52110,83.28580,0.59060,0.34190,0.78400
2025-03-31,68.666237,0.993952,0.35440,0.78040,,12.36280,5.93550,99.56450,20.95710,99.54210,...,0.15960,37.18160,47.42530,,49.67660,49.67660,83.49280,0.85700,0.41010,0.73860
2024-12-31,83.712524,0.820262,-0.1665,0.3395,4.65970,11.91020,5.96180,85.71980,19.35370,93.03670,...,0.18150,32.84000,41.36440,,42.77470,42.77470,84.80930,0.71640,0.38460,0.73970
2024-09-30,115.875969,0.722432,1.0819,1.4961,,15.87510,6.86920,41.34390,22.65230,100.49800,...,0.17940,38.28440,48.21700,,47.42890,47.42890,84.14270,0.47270,0.29920,0.93670
2024-06-30,138.393036,0.837296,1.2922,1.6284,,12.21590,5.42800,35.16250,17.81870,100.67150,...,0.18430,29.45930,37.22010,,38.10460,38.10460,84.90450,0.50710,0.31440,0.94460
2024-03-31,124.489738,1.111682,0.19020,0.46620,,21.96410,8.49960,64.69660,25.68670,98.81560,...,0.21860,38.87890,48.84240,,48.73220,48.73220,84.82610,0.27220,0.14490,0.70300
2023-12-31,99.581093,1.250134,-0.0136,0.2867,3.43070,17.20750,6.95380,47.38130,20.52240,103.24320,...,0.20860,33.33300,41.91990,,40.63450,40.63440,84.79770,0.25340,0.16150,0.82310
2023-09-30,87.538963,1.137563,1.1464,1.3368,,19.90700,7.49010,60.31940,24.17220,94.95430,...,0.19570,38.27280,47.78230,,45.82420,45.82420,83.46190,0.28540,0.17650,0.82410
2023-06-30,77.516129,1.129300,0.7496,0.9428,,17.67980,6.92000,48.90750,21.47380,116.88560,...,0.19340,35.77900,44.66670,,43.99260,43.99260,85.53220,0.28410,0.17670,0.87830
